# Gaussian processes

In this tutorial, we explain how to obtain the functionality of other packages such as GaussianProcesses.jl for Gaussian processes in Julia with GeoStats.jl. Gaussian processes and Simple Kriging are essentially [two names for the same concept](https://en.wikipedia.org/wiki/Kriging). The derivation of Kriging estimators, however; does **not** require distributional assumptions.

[Matheron](https://en.wikipedia.org/wiki/Georges_Matheron) and other important geostatisticians have generalized Gaussian processes to random fields with locally-varying mean and for situations where the mean is unknown. GeoStats.jl includes Gaussian processes as a special case as well as other more practical Kriging variants.

In machine learning, Gaussian processes are usually explained in 1D, and we will do the same here.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "Plots", "GR", "Interact"]
    Pkg.add(pkg)
end

# make sure this tutorial is reproducible
srand(2000);

INFO: Package GeoStats is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed
INFO: Package Interact is already installed


## The data

Let's start by drawing a set of points in the real line, each with a known value:

In [2]:
using Plots
gr(size=(1000,400), xlim=(0,1), ylim=(0,1))

x = collect(linspace(0.1,.9,10))
z = rand(10)

scatter(x, z, label="data", size=(1000,400))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data

We will skip an important step known in geostatistics as *variogram modeling*, and will proceed to the estimation step directly in order to avoid having too many concepts in this tutorial. These concepts are likely new to practioners of machine learning and can be very useful in practice. For more information on variogram modeling, please check the other tutorials in this folder.

## Simple Kriging

In simple Kriging (or Gaussian process), we need to specify a mean and a covariance-type function in order to interpolate the stochastic process at unseen locations. We choose a covariance (or variogram) model in GeoStats.jl and interact with the parameters:

In [3]:
using GeoStats
using Interact

# discretize the line segment for plotting
n = 200
xs = linspace(0,1,n)

@manipulate for s=linspace(0.,.1), r=linspace(0.,.1)
    # some given mean
    process_mean = .5
    
    # spatial structure
    γ = GaussianVariogram(sill=s, range=r)
    
    # create Gaussian process
    simkrig = SimpleKriging(x', z, γ, process_mean)
    
    # estimate mean and variance
    μ = zeros(n)
    σ² = zeros(n)
    for (i,loc) in enumerate(xs)
        μ[i], σ²[i] = estimate(simkrig, [loc])
    end
    
    σ = sqrt.(abs.(σ²))
    
    scatter(x, z, label="data")
    plot!(xs, μ, ribbon=σ, fillalpha=.5, label="Gaussian process")
end

Interact.Options{:SelectionSlider,Float64}(1: "input" = 0.04897959183673469 Float64 , "s", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(3: "input-2" = 0.04897959183673469 Float64 , "r", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip02)" points="
39.3701,267.506 44.1578,267.249 48.9455,266.878 53.7332,266.352 58.521,265.614 63.3087,264.596 68.0964,263.209 72.8841,261.346 77.6718,258.88 82.4595,255.668 
 87.2473,251.563 92.035,246.421 96.8227,240.126 101.61,232.603 106.398,223.84 111.186,213.901 115.974,202.935 120.761,191.171 125.549,178.91 130.337,166.498 
 135.124,156.608 139.912,167.825 144.7,179.013 149.488,189.706 154.275,199.45 159.063,207.833 163.851,214.503 168.638,219.187 173.426,221.7 178.214,221.946 
 183.002,219.924 187.789,215.727 192.577,209.544 197.365,201.66 202.152,192.456 206.94,182.394 211.728,172.007 216.516,161.871 221.303,162.007 226.091,176.72 
 230.879,192.263 235.667,208.366 240.454,224.758 245.242,241.173 250.03,257.344 254.817,272.993 259.605,287.813 264.393,301.457 269.181,313.535 273.968,323.621 
 278.756,331.279 283.544,336.099 288.331,337.741 293.119,335.984 297.907,330.758 302.695,322.178 307.482,327.067 312.27,335.166 317.058,339.956 321.845,341.466 
 326.633,339.891 331.421,335.548 336.209,328.834 340.996,320.174 345.784,309.975 350.572,298.594 355.359,286.317 360.147,273.364 364.935,259.903 369.723,246.07 
 374.51,232 379.298,217.848 384.086,203.8 388.874,190.841 393.661,200.486 398.449,210.127 403.237,219.258 408.024,227.437 412.812,234.3 417.6,239.567 
 422.388,243.043 427.175,244.602 431.963,244.189 436.751,241.802 441.538,237.496 446.326,231.377 451.114,223.613 455.902,214.43 460.689,204.119 465.477,193.025 
 470.265,181.537 475.052,177.964 479.84,189.543 484.628,200.858 489.416,211.515 494.203,221.156 498.991,229.472 503.779,236.215 508.566,241.196 513.354,244.281 
 518.142,245.384 522.93,244.465 527.717,241.53 532.505,236.631 537.293,229.873 542.08,221.421 546.868,211.495 551.656,200.365 556.444,188.341 561.231,190.755 
 566.019,200.293 570.807,208.579 575.595,215.169 580.382,219.687 585.17,221.837 589.958,221.425 594.745,218.363 599.533,212.675 604.321,204.496 609.109,194.067 
 613.896,181.734 618.684,167.935 623.472,153.185 628.259,138.053 633.047,123.133 637.835,109.001 642.623,96.1812 647.41,107.165 652.198,120.189 656.986,133.901 
 661.773,147.664 666.561,160.835 671.349,172.804 676.137,183.036 680.924,191.093 685.712,196.659 690.5,199.543 695.287,199.691 700.075,197.181 704.863,192.211 
 709.651,185.089 714.438,176.212 719.226,166.031 724.014,155.027 728.801,150.038 733.589,161.415 738.377,172.194 743.165,181.907 747.952,190.095 752.74,196.341 
 757.528,200.3 762.316,201.723 767.103,200.475 771.891,196.544 776.679,190.05 781.466,181.236 786.254,170.472 791.042,158.232 795.83,145.077 800.617,131.622 
 805.405,118.501 810.193,106.322 814.98,109.115 819.768,122.782 824.556,137.43 829.344,152.526 834.131,167.508 838.919,181.822 843.707,194.951 848.494,206.443 
 853.282,215.92 858.07,223.094 862.858,227.773 867.645,229.861 872.433,229.367 877.221,226.4 882.008,221.174 886.796,213.995 891.584,205.247 896.372,195.373 
 901.159,205.53 905.947,217.205 910.735,227.839 915.522,237.236 920.31,245.272 925.098,251.904 929.886,257.169 934.673,261.171 939.461,264.068 944.249,266.049 
 949.037,267.31 953.824,268.04 958.612,268.403 963.4,268.532 968.187,268.527 972.975,268.457 977.763,268.365 982.551,268.274 987.338,268.197 992.126,268.136 
 992.126,108.531 987.338,108.616 982.551,108.744 977.763,108.936 972.975,109.223 968.187,109.651 963.4,110.282 958.612,111.203 953.824,112.523 949.037,114.378 
 944.249,116.922 939.461,120.314 934.673,124.705 929.886,130.217 925.098,136.917 920.31,144.8 915.522,153.771 910.735,163.64 905.947,174.12 901.159,184.842 
 896.372,193.068 891.584,180.107 886.796,166.942 882.008,153.86 877.221,141.117 872.433,128.936 867.645,117.51 862.858,107.015 858.07,97.6248 853.282,89.5271 
 848.494,

## Ordinary Kriging

An immediate generalization of Simple Kriging is Ordinary Kriging. In this case, we still have the assumption that the mean of the field is constant, but this time we treat it as an unknown. Below is the estimation with Ordinary Kriging where we only need to specify the covariance-type function:

In [4]:
using GeoStats
using Interact

# discretize the line segment for plotting
n = 200
xs = linspace(0,1,n)

@manipulate for s=linspace(0.,.1), r=linspace(0.,.1)
    # spatial structure
    γ = GaussianVariogram(sill=s, range=r)
    
    # create Gaussian process
    ordkrig = OrdinaryKriging(x', z, γ)
    
    # estimate mean and variance
    μ = zeros(n)
    σ² = zeros(n)
    for (i,loc) in enumerate(xs)
        μ[i], σ²[i] = estimate(ordkrig, [loc])
    end
    
    σ = sqrt.(abs.(σ²))
    
    scatter(x, z, label="data")
    plot!(xs, μ, ribbon=σ, fillalpha=.5, label="Gaussian process")
end

Interact.Options{:SelectionSlider,Float64}(7: "input-3" = 0.04897959183673469 Float64 , "s", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(9: "input-4" = 0.04897959183673469 Float64 , "r", 0.04897959183673469, "0.04897959183673469", Interact.OptionDict(DataStructures.OrderedDict("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict(0.0102041=>"0.01020408163265306",0.0183673=>"0.018367346938775512",0.0265306=>"0.026530612244897958",0.0877551=>"0.08775510204081632",0.0163265=>"0.0163265306122449",0.0693878=>"0.06938775510204082",0.0857143=>"0.08571428571428572",0.0306122=>"0.030612244897959183",0.0387755=>"0.03877551020408163",0.0367347=>"0.036734693877551024"…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip02)" points="
39.3701,255.449 44.1578,255.256 48.9455,254.976 53.7332,254.577 58.521,254.014 63.3087,253.231 68.0964,252.155 72.8841,250.693 77.6718,248.733 82.4595,246.146 
 87.2473,242.79 92.035,238.522 96.8227,233.212 101.61,226.76 106.398,219.115 111.186,210.294 115.974,200.387 120.761,189.565 125.549,178.072 130.337,166.211 
 135.124,156.628 139.912,167.9 144.7,178.911 149.488,189.25 154.275,198.531 159.063,206.412 163.851,212.613 168.638,216.919 173.426,219.189 178.214,219.356 
 183.002,217.424 187.789,213.473 192.577,207.661 197.365,200.226 202.152,191.489 206.94,181.855 211.728,171.796 216.516,161.844 221.303,161.988 226.091,176.532 
 230.879,191.753 235.667,207.424 240.454,223.329 245.242,239.264 250.03,255.019 254.817,270.365 259.605,285.031 264.393,298.688 269.181,310.944 273.968,321.353 
 278.756,329.441 283.544,334.746 288.331,336.872 293.119,335.534 297.907,330.61 302.695,322.171 307.482,327.019 312.27,334.902 317.058,339.333 321.845,340.389 
 326.633,338.322 331.421,333.512 336.209,326.411 340.996,317.491 345.784,307.187 350.572,295.87 355.359,283.817 360.147,271.224 364.935,258.215 369.723,244.873 
 374.51,231.272 379.298,217.508 384.086,203.715 388.874,190.841 393.661,200.388 398.449,209.764 403.237,218.5 408.024,226.207 412.812,232.58 417.6,237.4 
 422.388,240.523 427.175,241.868 431.963,241.403 436.751,239.131 441.538,235.095 446.326,229.37 451.114,222.076 455.902,213.384 460.689,203.523 465.477,192.78 
 470.265,181.497 475.052,177.953 479.84,189.379 484.628,200.384 489.416,210.616 494.203,219.772 498.991,227.605 503.779,233.926 508.566,238.594 513.354,241.512 
 518.142,242.615 522.93,241.863 527.717,239.24 532.505,234.764 537.293,228.49 542.08,220.522 546.868,211.021 551.656,200.201 556.444,188.33 561.231,190.715 
 566.019,200.048 570.807,207.982 575.595,214.123 580.382,218.15 585.17,219.83 589.958,219.024 594.745,215.692 599.533,209.889 604.321,201.762 609.109,191.548 
 613.896,179.567 618.684,166.215 623.472,151.955 628.259,137.296 633.047,122.77 637.835,108.903 642.623,96.1811 647.41,107.08 652.198,119.848 656.986,133.173 
 661.773,146.467 666.561,159.147 671.349,170.664 676.137,180.536 680.924,188.369 685.712,193.871 690.5,196.859 695.287,197.269 700.075,195.145 704.863,190.642 
 709.651,184.011 714.438,175.588 719.226,165.767 724.014,154.979 728.801,150.031 733.589,161.266 738.377,171.744 743.165,181.038 747.952,188.743 752.74,194.503 
 757.528,198.032 762.316,199.132 767.103,197.706 771.891,193.762 776.679,187.422 781.466,178.911 786.254,168.563 791.042,156.803 795.83,144.134 800.617,131.112 
 805.405,118.313 810.193,106.304 814.98,109.088 819.768,122.571 824.556,136.891 829.344,151.56 834.131,166.073 838.919,179.939 843.707,192.698 848.494,203.942 
 853.282,213.329 858.07,220.584 862.858,225.505 867.645,227.972 872.433,227.946 877.221,225.48 882.008,220.717 886.796,213.893 891.584,205.323 896.372,195.393 
 901.159,205.243 905.947,216.367 910.735,226.233 915.522,234.689 920.31,241.665 925.098,247.18 929.886,251.325 934.673,254.257 939.461,256.169 944.249,257.276 
 949.037,257.788 953.824,257.893 958.612,257.749 963.4,257.478 968.187,257.162 972.975,256.857 977.763,256.59 982.551,256.372 987.338,256.204 992.126,256.079 
 992.126,88.416 987.338,88.6834 982.551,89.0678 977.763,89.6113 972.975,90.3674 968.187,91.4037 963.4,92.8039 958.612,94.6688 953.824,97.1157 949.037,100.275 
 944.249,104.283 939.461,109.269 934.673,115.338 929.886,122.557 925.098,130.928 920.31,140.379 915.522,150.752 910.735,161.798 905.947,173.19 901.159,184.533 
 896.372,193.088 891.584,180.181 886.796,166.839 882.008,153.387 877.221,140.143 872.433,127.405 867.645,115.443 862.858,104.506 858.07,94.8284 853.282,86.634 
 8

# Other generalizations

- In GeoStats.jl, we can also have random fields with non-constant mean with Universal Kriging. In this method, the mean is a polynomial on the coordinates of the problem up to a certain degree specified by the user.

- All methods in GeoStats.jl accept general distance functions. We can for example utilze the Haversine distance to compute covariances between latitude/longitude coordinates directly.

- Other methods for co-estimation and co-simulation will be available in future releases.